In [4]:
overpass_api_query = """
[out:json];
area[name="秋田県"]->.a;
(
  nwr["leisure"="park"](area.a);
);
out geom;
"""

import requests

overpass_api_res = requests.post("https://overpass-api.de/api/interpreter" , data = {"data": overpass_api_query})
overpass_api_json = overpass_api_res.json()

elements = []

for element in overpass_api_json['elements']:
    if 'geometry' in element:
        elements.append(element)

print(len(elements))

2271


In [5]:
import psycopg2

# PostgreSQLの情報
host = 'postgis'
port = '5432'
user = 'postgres'
password = 'postgres'
database = 'sandbox'

# PostgreSQLに接続
conn = psycopg2.connect(host=host, port=port, database=database, user=user, password=password)

# カーソルを作成
cursor = conn.cursor()

# テーブルを作成
cursor.execute('CREATE TABLE parks_akita_2023_07_18 (id BIGINT, name VARCHAR, name_en VARCHAR, geom GEOMETRY(Polygon, 4326));')

# 公園データを挿入
for element in elements:
    element_id = element['id']
    element_name = element['tags'].get('name', '').replace("'", "''")
    element_name_en = element['tags'].get('name:en', '').replace("'", "''")

    coordinates = element['geometry']

    # Polygonではないelementはスキップする
    if len(coordinates) < 3:
        continue

    # 座標データをPostGISの形式に変換
    polygon = 'POLYGON(('
    for coord in coordinates:
        lon = coord['lon']
        lat = coord['lat']
        polygon += f'{lon} {lat},'
    polygon = polygon.rstrip(',')  # 末尾のカンマを削除
    polygon += '))'

    # テーブルに挿入
    insert_query = f"INSERT INTO parks_akita_2023_07_18 (id, name, name_en, geom) VALUES ({element_id}, '{element_name}', '{element_name_en}', ST_SetSRID(ST_GeomFromText('{polygon}'), 4326));"
    print(insert_query)
    cursor.execute(insert_query)

# コミットして変更を確定
conn.commit()

# 接続を閉じる
conn.close()

INSERT INTO parks_akita_2023_07_18 (id, name, name_en, geom) VALUES (38869780, '千秋公園', 'Senshu Park', ST_SetSRID(ST_GeomFromText('POLYGON((140.1257225 39.7218698,140.1256728 39.7217184,140.1256283 39.7217032,140.1253635 39.7216629,140.1252873 39.7215771,140.1252794 39.7215191,140.1252785 39.7214601,140.1253083 39.7214073,140.1253475 39.7213852,140.1253999 39.7213729,140.1254974 39.7213549,140.1257582 39.7213261,140.1257252 39.7211564,140.1257102 39.7210781,140.1256775 39.7209179,140.125655 39.7207739,140.1255966 39.7204229,140.1255586 39.720165,140.1255593 39.7200586,140.1255577 39.7200474,140.1254572 39.7193419,140.1251537 39.7193719,140.1251268 39.7192852,140.1251143 39.7192451,140.1251028 39.7192081,140.1250369 39.7192115,140.1249685 39.7191214,140.1246497 39.7192944,140.1244451 39.7197418,140.1241835 39.7200618,140.1240653 39.7201062,140.1239964 39.7201293,140.1236837 39.7201962,140.1234095 39.7202928,140.123251 39.7204012,140.1231919 39.7204152,140.123153 39.7204062,140.1230332 39

In [6]:
import psycopg2

# PostgreSQLの接続情報
host = 'postgis'
port = '5432'
user = 'postgres'
password = 'postgres'
database = 'sandbox'

# テーブル名
parks_table = 'parks_akita_2023_07_18'
flood_areas_table = 'flood_areas_akita_2023_07_18'

# PostgreSQLへの接続
conn = psycopg2.connect(host=host, port=port, database=database, user=user, password=password)

# カーソルを作成
cursor = conn.cursor()

# 浸水している病院の検索クエリ
query = f"""
SELECT p.id, p.name
FROM {parks_table} AS p
WHERE EXISTS (
    SELECT 1
    FROM {flood_areas_table} AS f
    WHERE ST_Intersects(p.geom, f.geometry)
);
"""

# クエリの実行
cursor.execute(query)

# 結果の表示
for row in cursor.fetchall():
    print(row)

# カーソルと接続をクローズ
cursor.close()
conn.close()


(375813088, '拠点第一街区公園')
(461633887, '広面高田街区公園')
(461635009, '第一街区公園')
(461635010, '広面谷地眼街区公園')
(499329732, '')
(533987511, '')
(533987512, '')
(533987515, '桜第二緑地')
(542700630, '広面小沼街区公園')
(542700631, '広面鬼頭街区公園')
(544111395, '手形街区公園')
(545163335, '長面第一街区公園')
(545163488, '広面鬼頭第二街区公園')
(545163705, '長面第二街区公園')
(545790941, '広面近隣公園')
(575376835, '')
(622171944, '広面野添街区公園')
(736862056, 'Playground')
(737019303, '鬼頭三角公園')
(737353424, '長面第三街区公園')
(742630465, '')
(775457360, '楢山明田近隣公園')
(818747014, '手形新栄町街区公園')
(818747023, '手形緑地')
(844092042, '')
(886213535, '')
(905674647, '')
(977464206, '横森第二児童遊園地')
